# Naive Bayes text classification


### import lib

In [44]:
import math
import csv
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from collections import Counter


## variables

In [45]:
dataSetFile = "Data/data_00.csv"
dataSet = None
dataSetText = list()
dataSetLabel = list()
stop = stopwords.words('english')
spets = ['in', 'to', 'http', 'off', 're', 'arsen']

Reading .csv file:

In [46]:
def readCsvFile():
    global dataSet
    with open(dataSetFile, 'r') as ds:
        reader = csv.reader(ds)
        dataSet = [row for row in reader]
        ds.close()
    return dataSet

In [47]:
readCsvFile()

[['Chinese in Beijing are Chinese arsen', '0'],
 ['The Chinese are Chinese in Shanghai', '0'],
 ['in Chinese the Arsen Macao', '0'],
 ['Tokyo in Japan Are Chinese', '1'],
 ['Chinese The in are Chinese arsen Chinese the Tokyo Are Japan', '0']]

### seperated text & label

In [48]:
def sepTextLabel():
    numDS = len(dataSet)
    
    for i in range (0, numDS):
        temp1 = str(dataSet[i][0])
        dataSetText.append(temp1)

        
    for i in range (0, numDS):
        temp2 = str(dataSet[i][1])
        dataSetLabel.append(temp2)
      
    return dataSetText, dataSetLabel

In [49]:
sepTextLabel()

(['Chinese in Beijing are Chinese arsen',
  'The Chinese are Chinese in Shanghai',
  'in Chinese the Arsen Macao',
  'Tokyo in Japan Are Chinese',
  'Chinese The in are Chinese arsen Chinese the Tokyo Are Japan'],
 ['0', '0', '0', '1', '0'])

## standartization data (lowercases, del "stop_words")

### lowercases

In [50]:
def lowercases():
    global dataSetText
    dataSetForLowerCases = (dataSetText)
    for i in range (0, len(dataSetForLowerCases),):
        dataSetText[i] = dataSetForLowerCases[i].lower()
    return dataSetText

In [51]:
lowercases() 

['chinese in beijing are chinese arsen',
 'the chinese are chinese in shanghai',
 'in chinese the arsen macao',
 'tokyo in japan are chinese',
 'chinese the in are chinese arsen chinese the tokyo are japan']

### del stop_words (& space)

In [52]:
def deleteStopWords():
    
    stopAllDic = stop + spets
    
    global dataSetText    
    stroke_ = ""
    w = 0
    while (w < 2):
        for i in range(0, len(dataSetText)):            
            stroke_i = nltk.sent_tokenize(dataSetText[i])
            for words in stroke_i:
                each_word = nltk.word_tokenize(words)
                for j in each_word:
                    lower_words = j.lower()               
                    stopwords_removed = [w for w in lower_words if not w in stopAllDic]
                    if lower_words in stopAllDic: 
                        each_word.remove(j)
            stroke_ = " ".join(each_word)
            dataSetText[i] = stroke_
        w += 1
    return dataSetText

In [53]:
deleteStopWords()

['chinese beijing chinese',
 'chinese chinese shanghai',
 'chinese macao',
 'tokyo japan chinese',
 'chinese chinese chinese tokyo japan']

### split 80/20

In [54]:
def splitDataSetText():

    etalonDataSetText = list()
    etalonDataSetLabel = list()
    testDataSetText = list ()
    testDataSetLabel = list()
    for i in range (0, len(dataSetText)):
        if ((i + 1) != 5):
            etalonDataSetText.append(dataSetText[i])
            etalonDataSetLabel.append(dataSetLabel[i])
        else:
            testDataSetText.append(dataSetText[i])
            testDataSetLabel.append(dataSetLabel[i])

    return etalonDataSetText, etalonDataSetLabel, testDataSetText, testDataSetLabel

In [55]:
splitDataSetText()

(['chinese beijing chinese',
  'chinese chinese shanghai',
  'chinese macao',
  'tokyo japan chinese'],
 ['0', '0', '0', '1'],
 ['chinese chinese chinese tokyo japan'],
 ['0'])

#### Number of classes & probability


In [56]:
def numOfClasses(lst):
    s = set()
    k = 0
    
    for i in lst:
        if i not in s:
            s.add(i)
            k += 1 
            
    classNumber = [0] * k        
    probabilityClasses = [0] * k
    kl = 0
    
    while kl < k:        
        for i in range (0, len(lst)):
            if (int(lst[i]) == kl):
                classNumber[kl] += 1
            
            probabilityClasses[kl] = classNumber[kl]/len(lst)
        kl +=1        
            
    return k, probabilityClasses, s

In [57]:
numOfClasses(splitDataSetText()[1])

(2, [0.75, 0.25], {'0', '1'})

### Считаем следующие вероятности (или делаем словари слов из всех доков)
количество слов в доке (и в сумме)

In [58]:
def numberWords(currentDataSet):
    numberWords = [0] * (numOfClasses(splitDataSetText()[1])[0])
    cDS = ["None"] * len(currentDataSet)
    eDSL = splitDataSetText()[1]
    
    k = numOfClasses(splitDataSetText()[1])[0]
    ck = 0
    totalNumberWords = 0
    
    while ck < k:
        for i in range (0, len(currentDataSet)):
                if (int(eDSL[i]) == ck):
                    cDS[i] = (currentDataSet[i]).split()
                    numberWords[ck] = numberWords[ck] + len(cDS[i])
        ck += 1
        
    for i in range (0, len(numberWords)):
        totalNumberWords = totalNumberWords + (numberWords[i])
    
    return numberWords, totalNumberWords, k

In [59]:
numberWords(splitDataSetText()[0])

([8, 3], 11, 2)

### Список слов в корпусе (в таблицу) и сразу вероятность текущего слова в каждом из классов
(делаю set, перебираю датасет по каждому слову из set, создаю массив(словарь) слово+его вероятность

In [60]:
def tableOfWordsAndParam():
   
    tableOfWords = set()
    k = numberWords(splitDataSetText()[0])[2]
    masLabel = splitDataSetText()[1]
    summOfWords = numberWords(splitDataSetText()[0])[1]
    
    for i in (splitDataSetText()[0]):
        doc = i.split()
        for j in doc:
            tableOfWords.add(j)
     
    unicWords = list(tableOfWords) 
    totalWordsInEtalon = len (unicWords)
    masLabel = (splitDataSetText()[1]).copy()
    tableOfWordsAndProbInClass = []
    for i in range (0, totalWordsInEtalon):
        tableOfWordsAndProbInClass.append([])
        for j in range (0, (1+k+k)):
            tableOfWordsAndProbInClass[i].append(0)

    for i in range (0, totalWordsInEtalon):
        tableOfWordsAndProbInClass[i][0] = unicWords[i]
    
    for i in range (0, len(tableOfWords)): 
        w = 0
        w1 = 0
        kl = 0 
        
        jj = 0
        for j in (splitDataSetText()[0]):
            if (kl == int(masLabel[jj])):
                w += j.count(unicWords[i])
                tableOfWordsAndProbInClass[i][(int(masLabel[kl])+1)] = w
                jj += 1
                
            else:
                w1 += j.count(unicWords[i])
                tableOfWordsAndProbInClass[i][(int(masLabel[kl])+3)] = w1
                jj += 1
                

    return tableOfWordsAndProbInClass, unicWords, totalWordsInEtalon

In [61]:
 tableOfWordsAndParam()

([['chinese', 5, 0, 1, 0],
  ['shanghai', 1, 0, 0, 0],
  ['japan', 0, 0, 1, 0],
  ['macao', 1, 0, 0, 0],
  ['beijing', 1, 0, 0, 0],
  ['tokyo', 0, 0, 1, 0]],
 ['chinese', 'shanghai', 'japan', 'macao', 'beijing', 'tokyo'],
 6)

### Create NaiveBayes class

In [62]:
class NaiveBayes:
    
    tableOfWordsAndProbInClass = tableOfWordsAndParam()[0]
    test = splitDataSetText()[2]
    unicWords = tableOfWordsAndParam()[1]
    
    alfa = 1
    p1 = ((numOfClasses(splitDataSetText()[1])[1])[0])
    p2 = ((numOfClasses(splitDataSetText()[1])[1])[1])
    
    denominator1 = alfa * len(unicWords) + ((numberWords(splitDataSetText()[0])[0])[0])
    denominator2 = alfa * len(unicWords) + ((numberWords(splitDataSetText()[0])[0])[1])
    
    for i in range (0, len(test)):
        wordTest = test[i].split()
        for ii in range (0, len(unicWords)):
            for word in wordTest:
                if (word == unicWords[ii]):
                    p1 = p1 * ((alfa+tableOfWordsAndProbInClass[ii][1])/denominator1)
                    p2 = p2 * ((alfa+tableOfWordsAndProbInClass[ii][3])/denominator2)
                    
    if (p1 > p2):
        sp = 0
    else:
        sp = 1
        
    
    answer = [splitDataSetText()[2], sp]    
    print ("Pobability: \t'0:'", p1, "\t'1:'", p2)
    print ("or log: \t'0:'", math.log(p1), "\t'1:'", math.log(p2))
    print ("\nreturn: ", answer)


Pobability: 	'0:' 0.00030121377997263036 	'1:' 0.00013548070246744226
or log: 	'0:' -8.107690312843909 	'1:' -8.906681345001262

return:  [['chinese chinese chinese tokyo japan'], 0]
